# MCLabs Churn Analyzer - Data Preparation
Author: @cmh02

This Jupyter Notebook will be used for general data preparation for the model in three main steps:
1) Anonymizing the data
2) Combining the data
3) Cleaning the data

In [1]:
'''
MODULE/PACKAGE IMPORTS
'''

# System
import os
import re
import hashlib
from glob import glob
from dotenv import load_dotenv
from datetime import datetime, timedelta

# Data
import numpy as np
import pandas as pd

# Output/Display
from tqdm import tqdm

In [2]:
'''
ENVIRONMENT VARIABLES
'''

# Load environment file using python-dotenv
load_dotenv(dotenv_path="../env/.env")

# Load environmental variables
MCA_PEPPERKEY = os.getenv("MCA_PEPPERKEY")

# Ensure environmental variables are set
if not MCA_PEPPERKEY:
	raise ValueError("Missing required environment variable: MCA_PEPPERKEY")

In [3]:
'''
DATA ANONYMIZATION

To protect player privacy, the first portion of our data preparation is anonymizing our data. 
We will simply take the datafiles in gatheringoutput and replace the UUID field with a hashed
version of itself. Then we can save a private version, including these hashes, for our usage,
along with a public version, with no UUID's or hashes, for external analysis.
'''

# Define input and output folder paths
folderPath_gatheringoutput = "../data/gatheringoutput/"
folderPath_anonoutput_public = "../data/anonymized/public/"
folderPath_anonoutput_private = "../data/anonymized/private/"

# Create output folders if they don't exist
if not os.path.exists(folderPath_anonoutput_public):
	os.makedirs(folderPath_anonoutput_public, exist_ok=True)
if not os.path.exists(folderPath_anonoutput_private):
	os.makedirs(folderPath_anonoutput_private, exist_ok=True)

# Get the names of all gatheringoutput files using glob
gatheringOutputFiles = glob(os.path.join(folderPath_gatheringoutput, "**", "*.csv"), recursive=True)

# Iterate through the files
for filePath in tqdm(iterable=gatheringOutputFiles, desc="Anonymizing Data Files", unit="file"):
	# Read the CSV file
	df = pd.read_csv(filePath)

	# Anonymize the UUID column (UUID -> hash(PEPPER + UUID))
	df['UUID'] = [hashlib.sha256(f"{MCA_PEPPERKEY}:{uuid}".encode()).hexdigest() for uuid in df['UUID']]

	# Get relative path for gatheringoutput file location
	dataRelativeFilePath = os.path.relpath(filePath, folderPath_gatheringoutput)
	
	# Create private output path and save dataframe to path
	outputFilePath = os.path.join(folderPath_anonoutput_private, dataRelativeFilePath)
	os.makedirs(os.path.dirname(outputFilePath), exist_ok=True)
	df.to_csv(outputFilePath, index=False)
	
	# Drop the UUID column
	df.drop(columns=['UUID'], inplace=True)
	
	# Create public output path and save dataframe to path
	outputFilePath = os.path.join(folderPath_anonoutput_public, dataRelativeFilePath)
	os.makedirs(os.path.dirname(outputFilePath), exist_ok=True)
	df.to_csv(outputFilePath, index=False)


Anonymizing Data Files: 100%|██████████| 1/1 [00:00<00:00,  9.83file/s]


In [6]:
'''
DATA CLEANING

This cell will clean the data to accomplish a variety of tasks:
- For any missing values, the cell will be replaced with NULL values
- All <none> values will be turned into formal NULL values
- Players missing a last-seen date will be dropped
- Players missing other features will be filled with default values
'''

# Utility function for converting between Plan's date syntax and seconds-since-date
def planDateToSecondsSince(planDateString: str, untilUnixTimeStamp: int) -> int:
	# Convert the unix timestamp into a datetime object
	untilDateTime = datetime.fromtimestamp(untilUnixTimeStamp)

	# Check if the plan date is in weekday formatting or date formatting
	if re.match(r"^\S+ \S+ \S+ \S+:\S+$", planDateString):
		
		# If we already have date formatting, just parse the string
		planDateTime = datetime.strptime(planDateString, "%b %d %Y %H:%M")

	else:

		# If we have relative day-of-week formatting, then we need to figure out which day it is
		planDateStringSplit = planDateString.split(" ")
		planDayOfWeek = planDateStringSplit[0]
		planTimeOfDay = planDateStringSplit[1]

		# If the day is "Today", then it is really the day before recording
		if planDayOfWeek == "Today":
			planDateTime = untilDateTime - timedelta(days=1)
		
		# If the day is "Yesterday", then it is 2 days before
		elif planDayOfWeek == "Yesterday":
			planDateTime = untilDateTime - timedelta(days=2)

		# If the day is shown as a day-of-the-week, then calc difference + 1
		elif planDayOfWeek in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]:

			# Get week of the day as datetime
			planDateTime = datetime.strptime(planDayOfWeek, "%A")

			# Calculate difference in days then add 1 for offset
			relativeDaysPassed = (untilDateTime.weekday() - planDateTime.weekday()) % 7
			planDateTime = untilDateTime - timedelta(days=relativeDaysPassed + 1)

		# Detect incorrect formatting
		else:
			raise ValueError(f"Unrecognized plan date format: {planDateString}")

		# Set the time
		planDateTime = planDateTime.replace(hour=int(planTimeOfDay.split(":")[0]), minute=int(planTimeOfDay.split(":")[1]))

	# Return difference of the two dates in seconds
	return abs(int((planDateTime - untilDateTime).total_seconds()))

# Define input and output folder paths
folderPath_anonymized_private = "../data/anonymized/private/"
folderPath_cleaned_public = "../data/cleaned/public/"
folderPath_cleaned_private = "../data/cleaned/private/"

# Create output folders if they don't exist
if not os.path.exists(folderPath_cleaned_public):
	os.makedirs(folderPath_cleaned_public, exist_ok=True)
if not os.path.exists(folderPath_cleaned_private):
	os.makedirs(folderPath_cleaned_private, exist_ok=True)
	
# Get the names of all combined files using glob
combinedOutputFiles = glob(os.path.join(folderPath_anonymized_private, "**", "*.csv"), recursive=True)

# Clean each file
for filePath in tqdm(iterable=combinedOutputFiles, desc="Cleaning Data Files", unit="file"):

	# Load the file into a DataFrame
	df = pd.read_csv(filePath)

	# Replace <none> with NULL
	df.replace("<none>", pd.NA, inplace=True)
	
	# Replace "-" with NULL
	df.replace("-", pd.NA, inplace=True)

	# Fill missing values for other features
	df.fillna({
		"mcmmo_power_level": 0,
		"mcmmo_skill_ACROBATICS": 0,
		"mcmmo_skill_ALCHEMY": 0,
		"mcmmo_skill_ARCHERY": 0,
		"mcmmo_skill_AXES": 0,
		"mcmmo_skill_CROSSBOWS": 0,
		"mcmmo_skill_EXCAVATION": 0,
		"mcmmo_skill_FISHING": 0,
		"mcmmo_skill_HERBALISM": 0,
		"mcmmo_skill_MACES": 0,
		"mcmmo_skill_MINING": 0,
		"mcmmo_skill_REPAIR": 0,
		"mcmmo_skill_SALVAGE": 0,
		"mcmmo_skill_SMELTING": 0,
		"mcmmo_skill_SWORDS": 0,
		"mcmmo_skill_TAMING": 0,
		"mcmmo_skill_TRIDENTS": 0,
		"mcmmo_skill_UNARMED": 0,
		"mcmmo_skill_WOODCUTTING": 0,
		"lw_rev_total": 0,
		"lw_rev_phase": 0,
		"chemrank": 0,
		"policerank": 0,
		"donorrank": 0,
		"goldrank": 0,
		"current_month_votes": 0,
		"plan_player_time_total_raw": 0,
		"plan_player_time_month_raw": 0,
		"plan_player_time_week_raw": 0,
		"plan_player_time_day_raw": 0,
		"plan_player_time_afk_raw": 0,
		"plan_player_latest_session_length_raw": 0,
		"plan_player_favorite_server": "Spawn",
		"plan_player_sessions_count": 1,
		"leaderboard_position_chems_all": 0,
		"leaderboard_position_chems_week": 0,
		"leaderboard_position_police_all": 0,
		"leaderboard_position_police_week": 0,
		"balance": 0
	}, inplace=True)
	 
	# Drop players missing a last-seen date
	df.dropna(subset=["plan_player_lastseen"], inplace=True)

	# Convert last seen times into seconds since last seen
	recordingTimestamp = float(os.path.basename(os.path.dirname(filePath)))
	df["plan_player_lastseen"] = df["plan_player_lastseen"].apply(lambda x: planDateToSecondsSince(x, recordingTimestamp))

	# Remove any extra text from balance column
	df["balance"] = df["balance"].replace({"dollars": "", "dollar": "", "money": "", "Dollars": "", "Dollar": "", "Money": ""}, regex=True)

	# Get relative path for output file
	dataRelativeFilePath = os.path.relpath(filePath, folderPath_anonymized_private)
	directoryPath = os.path.dirname(dataRelativeFilePath)
	dataRelativeFilePath = os.path.join(directoryPath, "cleaned.csv")

	# Create private output path and save combined data
	outputFilePath = os.path.join(folderPath_cleaned_private, dataRelativeFilePath)
	os.makedirs(os.path.dirname(outputFilePath), exist_ok=True)
	df.to_csv(outputFilePath, index=False)

	# Drop the UUID column
	df.drop(columns=["UUID"], inplace=True)

	# Create public output path and save combined data
	outputFilePath = os.path.join(folderPath_cleaned_public, dataRelativeFilePath)
	os.makedirs(os.path.dirname(outputFilePath), exist_ok=True)
	df.to_csv(outputFilePath, index=False)


Cleaning Data Files: 100%|██████████| 1/1 [00:00<00:00,  8.48file/s]


In [ ]:
'''
FEATURE CREATION

Throughout the project, we may create several features that are derivations of raw features.
'''

# Define input and output folder paths
folderPath_cleaned_private = "../data/cleaned/private/"
folderPath_featurized_private = "../data/featurized/private/"
folderPath_featurized_public = "../data/featurized/public/"

# Create output folders if they don't exist
if not os.path.exists(folderPath_featurized_public):
	os.makedirs(folderPath_featurized_public, exist_ok=True)
if not os.path.exists(folderPath_featurized_private):
	os.makedirs(folderPath_featurized_private, exist_ok=True)

# Get the names of all cleaned files using glob
cleanedOutputFiles = glob(os.path.join(folderPath_cleaned_private, "**", "*.csv"), recursive=True)

# Create features
for filePath in tqdm(iterable=cleanedOutputFiles, desc="Featurizing Data Files", unit="file"):

	# Read the cleaned data
	df = pd.read_csv(filePath)

	# Derived Feature: Relative playtime between total and month (how much of the playtime was this month)
	df["plan_player_relativePlaytime_totalmonth"] = df["plan_player_time_total_raw"].astype(float) / df["plan_player_time_month_raw"].astype(float)

	# Derived Feature: Relative playtime between week and month (how much of the month was this week)
	df["plan_player_relativePlaytime_weekmonth"] = df["plan_player_time_week_raw"].astype(float) / df["plan_player_time_month_raw"].astype(float)

	# Derived Feature: Relative playtime between day and week (how much of the week was this day)
	df["plan_player_relativePlaytime_dayweek"] = df["plan_player_time_day_raw"].astype(float) / df["plan_player_time_week_raw"].astype(float)

	# Fix missing / infinities
	df.replace([np.inf, -np.inf], np.nan, inplace=True)
	df.fillna(0, inplace=True)

	# Get relative path for output file
	dataRelativeFilePath = os.path.relpath(filePath, folderPath_cleaned_private)
	directoryPath = os.path.dirname(dataRelativeFilePath)
	dataRelativeFilePath = os.path.join(directoryPath, "featurized.csv")

	# Create private output path and save combined data
	outputFilePath = os.path.join(folderPath_featurized_private, dataRelativeFilePath)
	os.makedirs(os.path.dirname(outputFilePath), exist_ok=True)
	df.to_csv(outputFilePath, index=False)

	# Drop the UUID column
	df.drop(columns=["UUID"], inplace=True)

	# Create public output path and save combined data
	outputFilePath = os.path.join(folderPath_featurized_public, dataRelativeFilePath)
	os.makedirs(os.path.dirname(outputFilePath), exist_ok=True)
	df.to_csv(outputFilePath, index=False)


Featurizing Data Files:   0%|          | 0/1 [00:00<?, ?file/s]


KeyError: ' plan_player_time_total_raw'

In [ ]:
'''
TARGET VARIABLE CREATION

This cell will create the target variable (churn) for the model.
'''

# Define input and output folder paths
folderPath_featurized_private = "../data/featurized/private/"
folderPath_targetted_public = "../data/targetted/public/"
folderPath_targetted_private = "../data/targetted/private/"

# Create output folders if they don't exist
if not os.path.exists(folderPath_targetted_public):
	os.makedirs(folderPath_targetted_public, exist_ok=True)
if not os.path.exists(folderPath_targetted_private):
	os.makedirs(folderPath_targetted_private, exist_ok=True)

# Get the names of all featurized files using glob
featurizedOutputFiles = glob(os.path.join(folderPath_featurized_private, "**", "*.csv"), recursive=True)

# For now, we will say that < 14 days means active, else churn
for filePath in tqdm(iterable=featurizedOutputFiles, desc="Targettizing Data Files", unit="file"):

	# Read the featurized data
	df = pd.read_csv(filePath)

	# Create the target variable
	df["churn"] = df["plan_player_lastseen"].apply(lambda x: 1 if x >= 1209600 else 0)

	# Get relative path for output file
	dataRelativeFilePath = os.path.relpath(filePath, folderPath_featurized_private)
	directoryPath = os.path.dirname(dataRelativeFilePath)
	dataRelativeFilePath = os.path.join(directoryPath, "targetted.csv")

	# Create private output path and save combined data
	outputFilePath = os.path.join(folderPath_targetted_private, dataRelativeFilePath)
	os.makedirs(os.path.dirname(outputFilePath), exist_ok=True)
	df.to_csv(outputFilePath, index=False)

	# Drop the UUID column
	df.drop(columns=["UUID"], inplace=True)

	# Create public output path and save combined data
	outputFilePath = os.path.join(folderPath_targetted_public, dataRelativeFilePath)
	os.makedirs(os.path.dirname(outputFilePath), exist_ok=True)
	df.to_csv(outputFilePath, index=False)


Targettizing Data Files: 100%|██████████| 1/1 [00:00<00:00, 10.81file/s]
